### Import standard modules

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

### Import NiPy modules

If you've installed python via miniconda, most packages are available by running `conda install package` in the terminal. In general, if a package is available through conda that's the easiest way to install it. Sometimes packages aren't available through conda, in which case you can use the python package installer, `pip`. If you're running this tutorial on `syzygy.ca`, conda is not installed so all packages are installed through pip. 

In [ ]:
#!pip install --user nipy nibabel dipy nilearn nipype

In [ ]:
import nipy
import nibabel as nib
import dipy
import nilearn
import nipype

### Load up some sample data

First, we'll load up the MNI T1 template as "t1img". `nib.load()` returns an image object. The exact structure of the image object will depend on what type of file you load. Mostly you're

Read more about nibabel image objects here: http://nipy.org/nibabel/nibabel_images.html

Loading the image object is fast because it doesn't load the image data automatically. This lets you quickly access header information if that's all you want. To load the image data as a numpy array just call `get_data()`

### Visualizing a slice is easy

### Note: Choose colormaps carefully! 

https://matplotlib.org/users/colormaps.html

### t1data and maskdata are both just numpy 3D arrays, so we perform normal array operations

## Exercise (optional): Parse an atlas xml file

* Parse the xml file using xml.etree.ElementTree
* Determine how to access the index and text of each element
* Loop over all the elements and place the data into a dictionary, with the label text as key and the index as value
* Plot an ROI by calling it by it's string, ie 'Body of corpus callosum'
* Evaluate the mean T1 in that ROI, again calling it by name

## Saving modified images as Nifti files
Let's make a copy of t1data and zero out voxels where maskdata==48

### Save the modified image

Create a new nifti image object with `nib.Nifti1Image()`. It needs a image array and an affine matrix

### Nibabel has tools for opening multiple formats including PAR/REC
Loading a PAR/REC image with nibabel and then saving to nifti is, in my experience, more robust than dcm2nii

## Exercise (optional): Make a  pandas dataframe with the mean values for each ROI
* Calculate the mean of each roi in the JHU atlas. Put in in a list or a dict
* Create a new `pd.DataFrame()` with the list/dict as the data, the mask labels as the index, and a relevant string as the column header 

### Not very "pythonic". Avoid loops when you can do it in one line!

Do this for multiple subjects and you've started building a database that you can run stats on, save to Excel, etc.

## Manipulating the affine
The affine matrix describes the relationship between the image matrix and the "real world" coordinate system. Image orientation manipulation should be approached with caution, but it's often necessary.

## Notice the difference?

Hard to spot, but the canonical oriented image is flipped left/right from the standard image. If you have a sagitally acquired image, the image will be reoriented in more dimensions.

## Exercise: Rotate the image and the affine matrix by 90 degrees. Save and open in FSL. Are the orientation labels correct?

http://nipy.org/nibabel/coordinate_systems.html

## Exercise: Tissue segmentation

http://nipy.org/dipy/examples_built/tissue_classification.html#example-tissue-classification

Let's run a tissue classification algorithm on our T1 image

## Nipype: Integrate imaging software into your Python code

https://miykael.github.io/nipype_tutorial/

Nipype is an amazing utility that allows you to link together different non-Python imaging tools within your Python analysis. It can be used simply to (for example) add FSL's eddy correction processing to you Python DTI processing; or it can be used to organize your entire processing workflow.

Personally I don't use the full workflow functionality. Instead I use it to call FSL and SPM functions from my Python scripts. It's much easier to write everything in pure Python instead of switching back and forth between FSL command line tools and SPM in Matlab. Nipype has an amazing number of interfaces, including:
* FSL
* SPM
* FreeSurfer
* niftireg
* ANTS
* AFNI
* Camino
* MIPAV
* DTK

Full list here: http://nipype.readthedocs.io/en/latest/documentation.html

For the Nipype interfaces to work, you just have to have the software you want to use install and certain environmental variables set so that Python knows where to look for the programs. For example,

None of these software tools are on the syzygy server. This example will only work on a computer with FSL installed.

`my_reg_file` is a string describing a path to a file. We can now use this variable as the input to another nipype interface, or we can load the nifti file into Python with the tools we learned previously.

## Exercise: DTI processing
http://nipy.org/dipy/examples_built/reconst_dti.html#example-reconst-dti

In [ ]:
import dipy.reconst.dti as dti
from dipy.data import read_stanford_hardi

The gtab (**g**radient **tab**le) is the object that holds the gradient information. This example gives it to us automatically. If you created your dti nifti file with dcm2nii, it probably also gave you  .bvec and .bval files. In that case, you could create the gtab with
```
from dipy.io import read_bvals_bvecs
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
from dipy.core.gradients import gradient_table
gtab = gradient_table(bvals, bvecs)
```

If you're reading data straight from a PAR/REC file, 
```
dtipar = nib.parrec.load(dti_par)
bvals, bvecs = dtipar.header.get_bvals_bvecs()
gtab = gradient_table(bvals, bvecs)
```

Create a TensorModel object with our gradient information

That's it! We can save the FA as a nifti, or do some numerical analysis on it with numpy, or make publication images, or whatever. 

## Exercise: Add an eddy correction step to the above pipeline using a Nipype interface

## Exercise: fMRI analysis with ICA

http://nilearn.github.io/connectivity/resting_state_networks.html